## Workflows

1. Update config.yaml –                                Yes
2. Update secrets.yaml [Optional] –                    No
3. Update params.yaml –                                Yes
4. Update the entity –                                 Yes
5. Update the configuration manager in src config –    Yes
6. Update the components –                             Yes
7. Update the pipeline –                               Yes
8. Update the main.py –                                No
9. Update the dvc.yaml –                               No

In [1]:
import os

In [2]:
%pwd

'd:\\mlops\\deep-lerning_project\\end_to_end_chest_classification_with_mlflow_and_dvc\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mlops\\deep-lerning_project\\end_to_end_chest_classification_with_mlflow_and_dvc'

## 1. Update config.yaml

### Write code in config.yaml file :

```yaml
artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion

## 3. Update params.yaml

* Note: When we will read `params.yaml` file then code will show Error to overcome this we add dummy value : `key: Value`

## 4. Update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## 5.Update the configuration manager in src config

### Write code in constants/__init__.py
```from pathlib import Path
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

### Note: Before importing write code 

* src\cnnClassifier\utils\common.py
* src\cnnClassifier\__init__.py

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

## 6.Update the components

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

### Define a function to download the data 

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## 7. Update the pipeline 

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-16 22:31:58,603: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-16 22:31:58,606: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 22:31:58,606: INFO: common: created directory at: artifacts]
[2025-03-16 22:31:58,606: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-16 22:31:58,611: INFO: 586056171: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=c0688ccd-ad8c-4a17-9b22-179a93d87b40
To: d:\mlops\deep-lerning_project\end_to_end_chest_classification_with_mlflow_and_dvc\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:12<00:00, 4.00MB/s]

[2025-03-16 22:32:17,044: INFO: 586056171: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
